In [1]:
import ast
import os
import clipboard
import re

def get_module_code(module_path):
    with open(module_path, 'r') as f:
        return f.read()

def get_all_imports(file_path):
    with open(file_path, 'r') as f:
        root = ast.parse(f.read())

    imports = [node for node in ast.walk(root) if isinstance(node, ast.Import)]
    import_froms = [node for node in ast.walk(root) if isinstance(node, ast.ImportFrom)]

    return imports, import_froms



def replace_import_with_code(file_path, project_root, imported_modules, non_project_imports):
    imports, import_froms = get_all_imports(file_path)

    code = get_module_code(file_path)

    for import_node in imports:
        for alias in import_node.names:
            original_name = alias.name
            as_name = alias.asname if alias.asname else alias.name
            module_path = os.path.join(project_root, original_name.replace('.', '/') + '.py')
            if os.path.isfile(module_path):
                if module_path not in imported_modules:
                    imported_modules.add(module_path)
                    module_code = replace_import_with_code(module_path, project_root, imported_modules, non_project_imports)
                    code = re.sub(f'import {original_name}', f'{module_code}', code)
                else:
                    code = re.sub(f'import {original_name}', f"{as_name} = {original_name}", code)
            else:
                non_project_imports.add(f'import {original_name} as {as_name}')
                code = re.sub(f'import {original_name} as {as_name}', "", code)
                code = re.sub(f'import {original_name}', "", code)

    for import_from_node in import_froms:
        module_path = os.path.join(project_root, import_from_node.module.replace('.', '/') + '.py')
        import_names = ", ".join([alias.name for alias in import_from_node.names])
        if os.path.isfile(module_path):
            if module_path not in imported_modules:
                imported_modules.add(module_path)
                module_code = replace_import_with_code(module_path, project_root, imported_modules, non_project_imports)
                code = re.sub(f'from {import_from_node.module} import {import_names}', f'{module_code}', code)
            else:
                code = re.sub(f'from {import_from_node.module} import {import_names}', "", code)
        else:
            for alias in import_from_node.names:
                original_name = alias.name
                as_name = alias.asname if alias.asname else alias.name
                non_project_imports.add(f'from {import_from_node.module} import {original_name} as {as_name}')
                code = re.sub(f'from {import_from_node.module} import {original_name} as {as_name}', "", code)
                code = re.sub(f'from {import_from_node.module} import {original_name}', "", code)

    code = clean_up_code(code)

    return code




def clean_up_code(code):
    # Remove any line that contains only a comma or whitespace,
    # or that appears to be a leftover part of an import statement.
    lines = code.split('\n')
    lines = [
        line for line in lines
        if not line.strip() in {',', ''}
        and not line.strip().startswith(',')
    ]
    return '\n'.join(lines)

def write_code_to_clipboard(code, non_project_imports, prompt):
    code_to_clipboard = prompt + "\n".join(non_project_imports) + "\n" + code
    clipboard.copy(code_to_clipboard)

def consolidate(project_root, input_file_path, prompt):
    imported_modules = set()
    non_project_imports = set()

    code = replace_import_with_code(input_file_path, project_root, imported_modules, non_project_imports)

    write_code_to_clipboard(code, non_project_imports, prompt)


In [62]:
implement_prompt = "Please implement the unimplemented metnods in the MatrixService. Also, please only output the implementations of these methods, I do not need all of the code repeated.\n\n"

In [63]:
consolidate("./", "./matrix/MatrixService.py", implement_prompt)

In [39]:
consolidate("./", "./pattern/Square.py", "Can you implement the rest of the passed methods.\n\n")

In [9]:
consolidate("./", "./test/PatternTest.py", "Please implement the methods in the class BlackburnsCheckmatePattern. Please note the commented descriptions of what these checkmates are.")

In [11]:
consolidate("./", "./pattern/checkmates/AnastasiasCheckmatePattern.py", "Based on the code in AnastasiasCheckmatePattern class, can you generate an english description of this checkmate?")

In [7]:
consolidate("./", "./TrainPPO.py", "")